# ChessML Netz

This neural network based on PyTorch will try to classifiy chess pieces from photos taken from the top of a chessboard.

In [1]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import random
import time
import matplotlib.pyplot as plt
import numpy as np

from torchvision import transforms
from PIL import Image

## Defining all possible classes

bb = Black Bishop
bk = Black King
bn = Black Knight
bp = Black Pawn
bq = Black Queen
br = Black Rook
...

In [2]:
CLASSES = ["bb", "bk", "bn", "bp", "bq", "br", "wb", "wk", "wn", "wp", "wq", "wr", "empty"]

## Define a normalization function for the analyzed data

The values are recommended by a PyTorch tutorial (https://youtu.be/32lHVbT09h8).

In [3]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    normalize
])

test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

## Defining the Neural Network

In [4]:
class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        # Defining the convolutional layers of the net
        self.conv1 = nn.Conv2d(3, 6, kernel_size=3)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=3)
        self.conv3 = nn.Conv2d(12, 24, kernel_size=3)
        self.conv4 = nn.Conv2d(24, 48, kernel_size=3)
        self.conv5 = nn.Conv2d(48, 96, kernel_size=3)

        # Defining dropout layer to prevent from memorizing
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.3)

        # Defining the fully connected layers of the net
        self.fc1 = nn.Linear(384, 64)  # 96*2*2 = 384
        self.fc2 = nn.Linear(64, 13)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)  # Relu because it's famous

        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv4(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv5(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.dropout1(x)

        x = x.view(-1, 384)  # Convert 2d data to 1d

        x = self.fc1(x)
        x = self.dropout2(x)
        x = F.relu(x)

        x = self.fc2(x)

        return torch.sigmoid(x)


## Training

In [5]:
def train(model, epoch, train_data, optimizer, criterion):
    model.train()    
    running_loss = 0.0
    for i, t_data in enumerate(train_data):
        data, target = t_data
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        out = model(data)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('Epoch: %2d [%d/%d] loss: %.3f' % (epoch, i, len(train_data), running_loss / 2000))
            running_loss = 0.0


## Validation

In [6]:
def validate(model, validation_data, criterion, epoch=0):
    model.eval()
    loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in validation_data:
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

            out = model(data)
            loss += criterion(out, target).item()
            _, prediction = torch.max(out.data, 1)
            total += target.size(0)

            if torch.cuda.is_available():
                correct += prediction.eq(target).sum().cpu().item()
            else:
                correct += prediction.eq(target).sum().item()

    loss = loss / total
    print("###################################")
    print("Epoch", epoch)
    print("Average loss:", loss)
    print("Accuracy: %.3f" % (100. * correct / total))
    print("###################################\n\n")


## Testing

In [7]:
def test(model, test_files):
    model.eval()
    orig_images = []
    images = []
    labels = []
    for i in range(4):
        file = random.choice(test_files)
        test_files.remove(file)
        img = Image.open(file[0])
        images.append(transform(img))
        orig_images.append(test_transform(img))
        labels.append(file[1])        
        
    images = torch.stack(images)
    orig_images = torch.stack(orig_images)
    
    out = model(images)
    
    _, predicted = torch.max(out, 1)

    imshow(torchvision.utils.make_grid(orig_images))
    print('   Actual: ', ' '.join('%5s' % CLASSES[labels[j]] for j in range(4)))
    print('Predicted: ', ' '.join('%5s' % CLASSES[predicted[j]] for j in range(4)))


## Helper functions

At first all available files are added to a file list. In addition the label of the file will be generated and added to the list.

In [8]:
def read_files(path):
    file_list = []
    for d in os.listdir(path):
        if os.path.isdir(os.path.join(path, d)):
            for f in os.listdir(os.path.join(path, d)):
                file_list.append([os.path.join(path, d, f), CLASSES.index(d)])
    return file_list

Generating tensors from the file list, we just created. All files get mixed up randomly.

In [9]:
def generate_tensors(file_list, batch_size):
    data = []
    data_list = []
    label_list = []
    files_count = len(file_list)
    for i in range(files_count):
        file = random.choice(file_list)
        file_list.remove(file)
        
        img = Image.open(file[0])  # Index 0: filename, Index 1: label
        img_tensor = transform(img)
        data_list.append(img_tensor)
        label_list.append(file[1])
        
        if len(data_list) >= batch_size:
            data.append((torch.stack(data_list), torch.tensor(label_list)))
            data_list = []
            label_list = []

            #Statistics
            print("Loaded batch", len(data), "of", int(files_count / batch_size))
            print("Percentage Done:", len(data) / int(files_count / batch_size) * 100, "%")
            if len(data) >= 1000:
                break
    return data


In [10]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


The save model function will save the state of a model after a specific epoch. 

In [11]:
def save_model(model, epoch):
    torch.save(model.state_dict(), "model/chess-net_{}.pt".format(epoch))
    print("Checkpoint saved")

## Main

Training and evaluating the ChessNet

In [12]:
def main():
    print("Loading Training Data")
    train_files = read_files("data/train_augmented")
    train_data = generate_tensors(train_files, 4)

    print("Loading Validation Data")
    validation_files = read_files("data/validation_augmented")
    validation_data = generate_tensors(validation_files, 4)
    
    print("Loading Test Data")
    test_files = read_files("data/validation_augmented")

    model = ChessNet()
    
    # Activate cuda support if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Check if model is already available
    # if os.path.isfile("model/chess-net.pt"):
    #    model = torch.load("model/chess-net.pt")

    # Defining the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    # Defining the loss function
    criterion = nn.CrossEntropyLoss()
    
    # Start training
    start = time.time()
    for epoch in range(1,5):
        train(model, epoch, train_data, optimizer, criterion)
        validate(model, validation_data, criterion, epoch)
        save_model(model, epoch)
    end = time.time()
    print("Training of the neuroal network done.")
    print("Time spent:", end-start, "s")
    
    # Testing the NN
    print("\nTest:")
    test(model, test_files)

In [13]:
if __name__ == "__main__":
    main()


Loading Training Data
Loaded batch 1 of 5650
Percentage Done: 0.017699115044247787 %
Loaded batch 2 of 5650
Percentage Done: 0.035398230088495575 %
Loaded batch 3 of 5650
Percentage Done: 0.05309734513274337 %
Loaded batch 4 of 5650
Percentage Done: 0.07079646017699115 %
Loaded batch 5 of 5650
Percentage Done: 0.08849557522123894 %
Loaded batch 6 of 5650
Percentage Done: 0.10619469026548674 %
Loaded batch 7 of 5650
Percentage Done: 0.12389380530973451 %
Loaded batch 8 of 5650
Percentage Done: 0.1415929203539823 %
Loaded batch 9 of 5650
Percentage Done: 0.1592920353982301 %
Loaded batch 10 of 5650
Percentage Done: 0.17699115044247787 %
Loaded batch 11 of 5650
Percentage Done: 0.19469026548672566 %
Loaded batch 12 of 5650
Percentage Done: 0.21238938053097348 %
Loaded batch 13 of 5650
Percentage Done: 0.2300884955752212 %
Loaded batch 14 of 5650
Percentage Done: 0.24778761061946902 %
Loaded batch 15 of 5650
Percentage Done: 0.2654867256637168 %
Loaded batch 16 of 5650
Percentage Done: 0.2

Loaded batch 133 of 5650
Percentage Done: 2.353982300884956 %
Loaded batch 134 of 5650
Percentage Done: 2.3716814159292032 %
Loaded batch 135 of 5650
Percentage Done: 2.3893805309734515 %
Loaded batch 136 of 5650
Percentage Done: 2.4070796460176993 %
Loaded batch 137 of 5650
Percentage Done: 2.4247787610619467 %
Loaded batch 138 of 5650
Percentage Done: 2.442477876106195 %
Loaded batch 139 of 5650
Percentage Done: 2.4601769911504423 %
Loaded batch 140 of 5650
Percentage Done: 2.47787610619469 %
Loaded batch 141 of 5650
Percentage Done: 2.495575221238938 %
Loaded batch 142 of 5650
Percentage Done: 2.5132743362831858 %
Loaded batch 143 of 5650
Percentage Done: 2.5309734513274336 %
Loaded batch 144 of 5650
Percentage Done: 2.5486725663716814 %
Loaded batch 145 of 5650
Percentage Done: 2.566371681415929 %
Loaded batch 146 of 5650
Percentage Done: 2.584070796460177 %
Loaded batch 147 of 5650
Percentage Done: 2.601769911504425 %
Loaded batch 148 of 5650
Percentage Done: 2.6194690265486726 %


Loaded batch 271 of 5650
Percentage Done: 4.79646017699115 %
Loaded batch 272 of 5650
Percentage Done: 4.814159292035399 %
Loaded batch 273 of 5650
Percentage Done: 4.831858407079646 %
Loaded batch 274 of 5650
Percentage Done: 4.849557522123893 %
Loaded batch 275 of 5650
Percentage Done: 4.867256637168142 %
Loaded batch 276 of 5650
Percentage Done: 4.88495575221239 %
Loaded batch 277 of 5650
Percentage Done: 4.902654867256637 %
Loaded batch 278 of 5650
Percentage Done: 4.920353982300885 %
Loaded batch 279 of 5650
Percentage Done: 4.938053097345133 %
Loaded batch 280 of 5650
Percentage Done: 4.95575221238938 %
Loaded batch 281 of 5650
Percentage Done: 4.9734513274336285 %
Loaded batch 282 of 5650
Percentage Done: 4.991150442477876 %
Loaded batch 283 of 5650
Percentage Done: 5.008849557522124 %
Loaded batch 284 of 5650
Percentage Done: 5.0265486725663715 %
Loaded batch 285 of 5650
Percentage Done: 5.04424778761062 %
Loaded batch 286 of 5650
Percentage Done: 5.061946902654867 %
Loaded bat

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/michaelwolz/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-972361fa1b80>", line 2, in <module>
    main()
  File "<ipython-input-12-98500db36b37>", line 4, in main
    train_data = generate_tensors(train_files, 4)
  File "<ipython-input-9-b54494146686>", line 10, in generate_tensors
    img = Image.open(file[0])  # Index 0: filename, Index 1: label
  File "/home/michaelwolz/Applications/anaconda3/lib/python3.7/site-packages/PIL/Image.py", line 2618, in open
    prefix = fp.read(16)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/michaelwolz/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'Keybo

KeyboardInterrupt: 